In [68]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, multilabel_confusion_matrix, confusion_matrix
from numpy import unravel_index

In [9]:
df =pd.read_csv("data/IRIS.csv")
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [19]:
X_all = df.iloc[:, 0:4]
y_all = df.iloc[:, 4:]
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.1)

In [57]:
cl = RandomForestClassifier(n_estimators=1)
cl.fit(X_train, y_train)
y_pred = cl.predict(X_test)
confusion_matrix(y_pred.tolist(), y_test.iloc[:, 0].tolist())

/Users/loreliegordon/.virtualenvs/sandbox2/lib/python3.7/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


array([[3, 0, 0],
       [0, 4, 0],
       [0, 0, 8]])

In [131]:
import pdb

In [177]:
class RandomForestFromScratch():


    def __init__(self, n_estimators=1, max_depth=3):
        self.max_depth = max_depth
        self.n_estimators = n_estimators
        pass

    def fit(self, X_train, y_train):
        self.X_n = len(X_train)
        self.d = len(X_train[0])
        self.weights = (1/self.X_n)*np.ones(self.X_n)


        for iter in range(self.n_estimators):

            choices = np.random.choice(range(self.X_n), p=self.weights, size=self.X_n)
            X_train_b = X_train[choices]
            y_train_b = y_train[choices]

            item_stack_fifo = []

            item_stack_fifo.append([X_train_b, y_train_b, 0, "start", None])

            max_iters = 100
            tot_count = 0

            root = None
            
            while item_stack_fifo and (tot_count < max_iters):

                tot_count += 1
                [X_train_b, y_train_b, depth, node_dir, parent] = item_stack_fifo.pop(0)

                n, _ = X_train_b.shape

                if n == 0:
                    print("Not adding a node here")
                    continue

                all_gina_scores = np.zeros((n, self.d))
                for dim in range(self.d):
                    for row in range(n):
                        all_gina_scores[row, dim] = self._calculate_score(X_train_b, y_train_b, dim, row)

                best_dim, best_split_val, best_split_val_index = self._choose_best_score(all_gina_scores)

                # print(f"best_dim {best_dim}, best_split_val {best_split_val}, best_split_val_index {best_split_val_index} ")

                # print(f"X_train_b.shape {X_train_b.shape} best_split_val_index {best_split_val_index}")
                decision_boundary = X_train_b[best_split_val_index, best_dim]
                
                # print(f"node_dir {node_dir} current_depth: {depth}, best_dim {best_dim} best_split_val {best_split_val} decision_boundary {decision_boundary}")
                

                node = TreeNode(dim=best_dim, val=decision_boundary, depth=depth)

                if parent is not None:
                    if node_dir == "left":
                        parent.left = node
                    else:
                        parent.right = node
                else:
                    root = node

                X_train_left_b = X_train_b[X_train_b[:, best_dim] <= decision_boundary, :]
                X_train_right_b = X_train_b[X_train_b[:, best_dim] > decision_boundary, :]
                y_train_left_b = y_train_b[X_train_b[:, best_dim] <= decision_boundary, :]
                y_train_right_b = y_train_b[X_train_b[:, best_dim] > decision_boundary, :]

                if (depth + 1) < self.max_depth:

                    item_stack_fifo.append([X_train_left_b, y_train_left_b, depth+1, "left", node])
                    item_stack_fifo.append([X_train_right_b, y_train_right_b, depth+1, "right", node])

            root.depth_first_print()

    def _calculate_score(self, X_train_b, y_train_b, dim, row):
        
        score = np.random.uniform(X_train_b.min(), X_train_b.max())
        return score

    def _choose_best_score(self, all_gina_scores):
        best_split_val_index, best_dim = unravel_index(all_gina_scores.argmax(), all_gina_scores.shape)
        
        best_split_val = all_gina_scores[best_split_val_index, best_dim]
        return (best_dim, best_split_val, best_split_val_index)

In [178]:
rf = RandomForestFromScratch(max_depth=3)
rf.fit(X_train.to_numpy(), y_train.to_numpy())

Node depth 0 self.dim 2 self.val 4.5
Node depth 1 self.dim 2 self.val 1.5
Node depth 2 self.dim 1 self.val 3.7
Node depth 2 self.dim 2 self.val 1.7
Node depth 1 self.dim 2 self.val 5.0
Node depth 2 self.dim 1 self.val 2.8
Node depth 2 self.dim 1 self.val 2.9


In [124]:
class TreeNode:
    
    def __init__(self, dim=0, val=0, left=None, right=None, depth=None, is_terminal=False, terminal_val=None):
        self.dim = dim
        self.val = val
        self.left = left
        self.right = right
        self.depth = depth
        self.is_terminal = is_terminal
        self.terminal_val = terminal_val

    def depth_first_print(self):
        print(f"Node depth {self.depth} self.dim {self.dim} self.val {self.val}")
        if self.left:
            self.left.depth_first_print()
        
        if self.right:
            self.right.depth_first_print()

    def predict(self, X):

        test_val = X[:, self.dim]
        if test_val >= self.val:
            return self.right(X)
        else:
            return self.left(X)

In [119]:
root = TreeNode(0, 0, depth=0)
root.left = TreeNode(1, 1, depth=1)
root.right = TreeNode(2,2, depth=1)

In [120]:
root.depth_first_print()

Node depth 0
Node depth 1
Node depth 1
